<a href="https://colab.research.google.com/github/DSarceno/DMRG/blob/main/Super_Hamiltonian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Super Hamiltoniano**
#### *Autor: Diego Sarceño*

El Super Hamiltoniano de una cadena de N partículas de Spin-$\frac{1}{2}$. Este se construye añadiendo partículas de dos en dos, al centro de un arreglo inicial de 2 partículas, las cuales solo interactuan a primeros vecinos. Con esta idea, el Super Hamiltoniano se divide en dos cadenas (izquierda y derecha) las cuales se unen al final, tomando en cuenta la contribución general.

## Observables 
#### Operador Proyección
- $S^z = \frac{\hbar}{2} \left(
\begin{array}{cc}
 1 & 0 \\
 0 & -1 \\
\end{array}
\right)$

#### Operadores Escalera
- $S^+ = \hbar \left(
\begin{array}{cc}
 0 & 1 \\
 0 & 0 \\
\end{array}
\right)$
- $S^- = \hbar \left(
\begin{array}{cc}
 0 & 0 \\
 1 & 0 \\
\end{array}
\right)$

Conforme se añaden partículas es necesario tomar en cuenta la contribución, dependiendo de donde se agregue la partícula. En caso de agregarse a la derecha de la cadena en la $i-$ésima posición
$$ \overset{\sim}{S} ^z _{L,i} = \mathbb{1} _{2^{i - 1}} \otimes S^z  $$
y
$$ \overset{\sim}{S} ^\pm _{L,i} = \mathbb{1} _{2^{i - 1}} \otimes S^\pm, $$
en caso contrario
$$ \overset{\sim}{S} ^z _{R,i} = S^z \otimes \mathbb{1} _{2^{i - 1}} $$
y
$$ \overset{\sim}{S} ^\pm _{R,i} = S^\pm \otimes \mathbb{1} _{2^{i - 1}}. $$

Dado que se esta trabajando con funciones recursivas, es necesario tener una condición inferior que detenga la función. En este caso es el hamiltoniano para un sistema de dos partículas
$$ H_2 = S^z \otimes S^z + \frac{1}{2} \left( S^+ \otimes S^- + S^- \otimes S^+ \right). $$

In [1]:
# Librerías a utlizar
import math as m
import numpy as np
import scipy.constants as sc


# Input (dimension de la mitad de la lista (super hamiltoniano), toda la lista (hamiltoniano))
n = 3

In [2]:
###       Definiendo Obserbables       ###

# Operadores asociados a partículas con spin 1/2
# Tomando \hbar = 1
sz = (1/2)*np.array([[1,0],[0,-1]]) # Spin projection operator in 'z'
sp = np.array([[0,1],[0,0]]) # Ladder operators
sm = np.array([[0,0],[1,0]])
H2 = np.kron(sz,sz) + (1/2)*(np.kron(sp,sm) + np.kron(sm,sp)) # Hamiltonian for 2 particles of spin-1/2


# Operadores para una partícula añadida a la derecha de una cadena de spins
def Sz(i):
    return np.kron(np.eye(2**(i-1)),sz)

def Sp(i):
    return np.kron(np.eye(2**(i-1)),sp)

def Sm(i):
    return np.kron(np.eye(2**(i-1)),sm)


# Operadores para una particula añadida a la izquierda de una cadena de spins
def SzR(i):
    return np.kron(sz, np.eye(2**(i - 1)))

def SpR(i):
    return np.kron(sp, np.eye(2**(i - 1)))

def SmR(i):
    return np.kron(sm, np.eye(2**(i - 1)))

## Bloques

Como se mencionó anteriormente, el Super Hamiltoniano se construye en base a un bloque izquierdo y derecho de igual longitud. Por lo que, se construye el Hamiltoniano de cada uno de los bloques.

### Bloque Izquierdo

$$ H_{L,i} = H_{L,i-1} \otimes \mathbb{1} _{2} + \overset{\sim}{S} ^z _{L,i - 1} \otimes S^z + \frac{1}{2} \left( \overset{\sim}{S} ^+ _{L,i - 1} \otimes S^- + \overset{\sim}{S} ^- _{L,i - 1} \otimes S^+ \right) $$

### Bloque Derecho

$$ H_{R, i} = \mathbb{1} _2 \otimes H_{R,i - 1} + S^z \otimes \overset{\sim}{S} ^z _{R,i - 1} + \frac{1}{2} \left( S^+ \otimes \overset{\sim}{S} ^- _{R,i - 1} + S^- \otimes \overset{\sim}{S} ^+ _{R,i - 1} \right) $$

In [3]:
# BLOQUE IZQUIERDO
def HL(i):
    if i > 2:
        return np.kron(HL(i - 1), np.eye(2)) + np.kron(Sz(i - 1), sz) + 0.5*(np.kron(Sp(i - 1), sm) + np.kron(Sm(i - 1), sp))
    elif i == 2:
        return H2
    else:
        return 1


# BLOQUE DERECHO
def HR(i):
    if i > 2:
        return np.kron(np.eye(2), HR(i - 1)) + np.kron(sz, SzR(i - 1)) + 0.5*( np.kron(sp, SmR(i - 1)) + np.kron(sm, SpR(i - 1)) )
    elif i == 2:
        return H2
    else:
        return 1

## Super Hamiltoniano

Combinando los dos bloques y las interacciones presentes, el Super Hamiltoniano es

$$ \mathcal{H}_i = \mathbb{1} _{2^i} \otimes H_{L,i} + H_{R,i} \otimes \mathbb{1} _{2^i} + \mathbb{1} _{2^{i - 1}} \otimes S^z \otimes S^z \otimes \mathbb{1} _{2^{i - 1}} + \frac{1}{2} \left( \mathbb{1} _{2^{i - 1}} \otimes S^+ \otimes S^- \otimes \mathbb{1} _{2^{i - 1}} + \mathbb{1} _{2^{i - 1}} \otimes S^- \otimes S^+ \otimes \mathbb{1} _{2^{i - 1}} \right). $$

In [4]:
# Combinando el bloque derecho e izquierdo
# super_hamiltonian (Eq. 2.35 Strongly Correlated Systems - A. Avella, F. Mancini)
SH = lambda d: np.kron(np.eye(2**d), HL(d)) + np.kron(HR(d), np.eye(2**d)) +\
    np.kron(np.kron(np.kron(np.eye(2**(d - 1)), sz), sz), np.eye(2**(d - 1))) +\
    0.5*( np.kron(np.kron(np.kron(np.eye(2**(d - 1)), sp), sm), np.eye(2**(d - 1))) +\
    np.kron(np.kron(np.kron(np.eye(2**(d - 1)), sm), sp), np.eye(2**(d - 1))) )

